<a href="https://colab.research.google.com/github/bankyadam/B-GY-INTAL1/blob/main/Kepbeszerzes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kepbeszerzes

Csatlakozzunk a Google Drive meghajtonkhoz (Hitelesites szukseges)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Ellenorizzuk az aktualis mappa tartalmat

In [2]:
!ls

gdrive	sample_data


Ezt kovetoen letrehozunk egy mappat, melyben dolgozni fogunk

In [4]:
import os
import shutil

folder_path = "/content/gdrive/MyDrive/B-VI-MESTIN/ImageScrapper/Images"

if (os.path.exists(folder_path)):
  shutil.rmtree(folder_path)

os.makedirs(folder_path, exist_ok=True)

Belepunk a mappankba

In [6]:
%cd /content/gdrive/MyDrive/B-VI-MESTIN/ImageScrapper/

/content/gdrive/MyDrive/B-VI-MESTIN/ImageScrapper


Telepitsuk a kepletolto alkalmazast

In [5]:
!pip install pexels-api

In [9]:
import argparse
import json
import os
import time

import requests
import tqdm
from pexels_api import API

PAGE_LIMIT = 5 # 20
RESULTS_PER_PAGE = 1; # 80

PEXELS_API_KEY = '1KdkaXiKlqV22Uk3UBts0UB5uZYyBMlH7n9AzwurwVVTFRrOiWFSvti4'

api = API(PEXELS_API_KEY)
query = 'pizza'

photos_dict = {}
page = 1
counter = 0

#Step 1 : getting urls and metadata
while page <= PAGE_LIMIT:
  api.search(query, page=page, results_per_page=RESULTS_PER_PAGE)
  photos = api.get_entries()
  for photo in tqdm.tqdm(photos):
    photos_dict[photo.id] = vars(photo)['_Photo__photo']
    counter += 1
    if not api.has_next_page:
      break
    page += 1
    time.sleep(2)

print(f"Finishing at page: {page}")
print(f"Images were processed: {counter}")

100%|██████████| 1/1 [00:02<00:00,  2.00s/it]

Finishing at page: 6
Images were processed: 5


A kereses utan toltsuk le a a mappankba

In [11]:
from PIL import Image

PATH = "/content/gdrive/MyDrive/B-VI-MESTIN/ImageScrapper/Images"
RESOLUTION = "original"
IMAGE_SIZE = (640, 640)

if photos_dict:
  os.makedirs(PATH, exist_ok=True)

  with open(os.path.join(PATH, f'{query}.json'), 'w') as fout:
    json.dump(photos_dict, fout)

  for val in tqdm.tqdm(photos_dict.values()):
    url = val['src'][RESOLUTION]
    fname = os.path.basename(val['src']['original'])
    image_path = os.path.join(PATH, fname)
    if not os.path.isfile(image_path):
      response = requests.get(url, stream=True)
      with open(image_path, 'wb') as outfile:
        outfile.write(response.content)

      with Image.open(image_path) as img:
        img = img.resize(IMAGE_SIZE)
        img.save(image_path)

    else:
      print(f"file {image_path} already exists")


100%|██████████| 5/5 [00:04<00:00,  1.13it/s]
